라이브러리 설치
- pip install python-pptx

In [8]:
from pptx import Presentation 
from pptx.util import Pt
import pandas as pd
import copy

In [9]:
df = pd.read_excel("./data/수험번호.xlsx")
prs = Presentation("./data/수험표_샘플.pptx")

names = iter(df['이름'])
nos = iter(df['수험번호'])
cnt = len(df)
print(cnt)

400


In [10]:
def copy_slide(prs, org_slide):
    copied_slide = prs.slides.add_slide(org_slide.slide_layout) # 슬라이드 추가 -  레이아웃 복사
    for shape in org_slide.shapes:  
        org_el = shape.element                                  # 원본 슬라이드의 모든 요소를 가져옴
        new_el = copy.deepcopy(org_el)                          # 요소를 복사
        copied_slide.shapes._spTree.insert_element_before(new_el, "p:extLst") # 복사한 요소를 슬라이드에 추가
    
    for value in org_slide.part.rels:                     # 원본 슬라이드의 모든 링크를 가져옴      
        if "notesSlide" not in value.reltype:             # notesSlide(노트)는 제외
            copied_slide.part.rels.get_or_add(            # 복사한 슬라이드에 링크를 추가
                value.reltype,                            # 링크 타입
                value._target                             # 링크 대상
                )
    return copied_slide

In [11]:
for i in range(int(cnt/2)):
    next_slide = copy_slide(prs, prs.slides[0])     # 첫번째 슬라이드 복사
    for shape in next_slide.shapes:                 # 복사한 슬라이드의 모든 요소를 가져옴
        if shape.shape_type == 17 and shape.text == "이름":   # 텍스트 상자이고, 텍스트가 "이름"인 경우
            shape.text_frame.paragraphs[0].text = next(names) # 텍스트를 이름으로 변경
            shape.text_frame.paragraphs[0].font.size = Pt(40)
            shape.text_frame.paragraphs[0].font.bold = True
            shape.text_frame.paragraphs[0].font.name = "나눔고딕"
        if shape.shape_type == 17 and shape.text == "수험번호":
            shape.text_frame.paragraphs[0].text = next(nos)
            shape.text_frame.paragraphs[0].font.size = Pt(40)
            shape.text_frame.paragraphs[0].font.bold = True
            shape.text_frame.paragraphs[0].font.name = "나눔고딕"

prs.save("수험표_결과.pptx")